In [2]:
import numpy as np
import tespy
from matplotlib import pyplot as plt

from tespy.networks import Network
from tespy.components import (Sink, Source, Valve, SimpleHeatExchanger, Compressor, CycleCloser)
from tespy.components import CombustionChamber # adiabatic ?
from tespy.components import (HeatExchanger, Condenser, Desuperheater)
from tespy.components import (Drum, Merge, Separator, Splitter)
from tespy.components import (Pump, Turbine)
from tespy.connections import Connection, Bus
from tespy.tools import ExergyAnalysis
from CoolProp.CoolProp import PropsSI as cpsi

In [4]:
#fluids
wf = 'Water'
fld_wf = {wf:1}

air = 'Air'
fld_air = {'N2': 0.79, 'O2': 0.21}

#network
dampferzeuger = Network(T_unit = 'C', p_unit = 'bar', h_unit = 'kJ / kg', m_unit = 't / h')

src_water = Source('Water_source')
snk_water = Sink('Water_sink')
comp_water = Compressor('Water_compressor')
economizer = HeatExchanger('Economizer')
evaporator = HeatExchanger('Evaporator')
drum = Drum('Trommel')

src_air = Source('Air_source')
snk_air = Sink('Air_sink')


c00 = Connection(src_water, 'out1', comp_water, 'in1', label = '00')
c01 = Connection(comp_water, 'out1', economizer, 'in2', label = '01')
c02 = Connection(economizer, 'out2', drum, 'in1', label = '02')
c03 = Connection(drum, 'out1', evaporator, 'in2', label = '03')
c04 = Connection(evaporator, 'out2', drum, 'in2', label = '04')
c05 = Connection(drum, 'out2', snk_water, 'in1', label = '05')

c10 = Connection(src_air, 'out1', evaporator, 'in1', label = '10')
c11 = Connection(evaporator, 'out1', economizer, 'in1', label = '11')
c12 = Connection(economizer, 'out1', snk_air, 'in1', label = '12')



dampferzeuger.add_conns(c00, c01, c02, c03, c04, c05, c10, c11, c12)

# parameter
c00.set_attr(T = 60, p = 1, m = 1350, fluid = fld_wf)
c01.set_attr(p = 41)

T_sat = cpsi('T', 'Q', 0, 'P', 41*1e5, wf) - 273.15
#c04.set_attr(T = T_sat, x = 1)

#h_sat_0 = cpsi('H', 'Q', 0, 'P', 41*1e5, wf)/1e3
h_sat_0 = cpsi('H', 'Q', 0, 'T', T_sat + 273.15, wf)/1e3
c02.set_attr(h = h_sat_0)

#h_sat_1 = cpsi('H', 'Q', 1, 'P', 41*1e5, wf)/1e3
h_sat_1 = cpsi('H', 'Q', 1, 'T', T_sat + 273.15, wf)/1e3
c05.set_attr(h = h_sat_1)




c10.set_attr(T = 1000, p = 30, m = 100000, fluid = fld_air)

comp_water.set_attr(eta_s = 0.8)
economizer.set_attr(pr1 = 1, pr2 = 1)
evaporator.set_attr(pr1 = 1)

motor_comp_water = Bus('motor-for-water-compression')
motor_comp_water.add_comps({'comp' : comp_water, 'char' : 0.975, 'base' : 'bus'})

#heat_economizer = Bus('heat-for-economizer')
#heat_economizer.add_comps({'comp' : economizer, 'base' : 'bus'})


#dampferzeuger.add_busses(motor_comp_water, heat_economizer)
dampferzeuger.add_busses(motor_comp_water)

#solve
dampferzeuger.solve(mode='design')
dampferzeuger.print_results()

Singularity in jacobian matrix, calculation aborted! Make sure your network does not have any linear dependencies in the parametrisation. Other reasons might be
-> given temperature with given pressure in two phase region, try setting enthalpy instead or provide accurate starting value for pressure.
-> given logarithmic temperature differences or kA-values for heat exchangers, 
-> support better starting values.
-> bad starting value for fuel mass flow of combustion chamber, provide small (near to zero, but not zero) starting value.



 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 2.36e+10   | 0 %        | NaN        | NaN        | NaN        | NaN        | NaN        
Total iterations: 1, Calculation time: 0.00 s, Iterations per second: 317.80

##### RESULTS (Bus: motor-for-water-compression) #####
+-------+-------------------+-------------+--------------+
|       |   component value |   bus value |   efficiency |
|-------+-------------------+-------------+--------------|
| total |         0.000e+00 |   0.000e+00 |          nan |
+-------+-------------------+-------------+--------------+


251.8226901958808